In [1]:
import numpy as np
import math
from calcul_structure import *

In [2]:
## Propriété materiaux [N, m]
E = 2*10**8
I = 2000*10**(-6)
A = 1000*10**(-6)

#Element 1
L1, L2 = 5, 6 

In [3]:
#Matrice de rigidité élémentaire en coordonnées locales
mat_elem_local_1 = matrice_elementaire(E, I, A, L1)
mat_elem_local_2 = matrice_elementaire(E, I, A, L2)
mat_elem_local_3 = matrice_elementaire(E, I, A, L1)

In [4]:
#Matrice de rotatio
rot_elem_1 = rotation_matrice(math.pi/2)
rot_elem_2 = rotation_matrice(0)
rot_elem_3 = rotation_matrice(3*math.pi/2)
print(rot_elem_3.astype(int))

[[ 0 -1  0  0  0  0]
 [ 1  0  0  0  0  0]
 [ 0  0  1  0  0  0]
 [ 0  0  0  0 -1  0]
 [ 0  0  0  1  0  0]
 [ 0  0  0  0  0  1]]


In [5]:
#Matrice de rigidité élémentaire en coordonnées locales
mat_elem_global_1 = np.matmul(np.matmul(np.transpose(rot_elem_1),mat_elem_local_1), rot_elem_1)
mat_elem_global_2 = np.matmul(np.matmul(np.transpose(rot_elem_2),mat_elem_local_2), rot_elem_2)
mat_elem_global_3 = np.matmul(np.matmul(np.transpose(rot_elem_3),mat_elem_local_3), rot_elem_3)

print(mat_elem_global_1.astype(int))

[[ 38400      0 -96000 -38400      0 -96000]
 [     0  40000      0      0 -40000      0]
 [-96000      0 320000  96000      0 160000]
 [-38400      0  96000  38400      0  96000]
 [     0 -40000      0      0  40000      0]
 [-96000      0 160000  96000      0 320000]]


In [9]:
#Assemblag de la matrice de rigidite
K_global = np.zeros((7, 7), dtype=np.float32)
K_global[0:4, 0:4] = K_global[0:4, 0:4] + mat_elem_global_1[2:6, 2:6]
K_global[1:7, 1:7] = K_global[1:7, 1:7] + mat_elem_global_2[0:6, 0:6]
K_global[4:7, 4:7] = K_global[4:7, 4:7] + mat_elem_global_3[0:3, 0:3]

print(K_global.astype(int))

[[320000  96000      0 160000      0      0      0]
 [ 96000  71733      0  96000 -33333      0      0]
 [     0      0  62222  66666      0 -22222  66666]
 [160000  96000  66666 586666      0 -66666 133333]
 [     0 -33333      0      0  71733      0  96000]
 [     0      0 -22222 -66666      0  62222 -66666]
 [     0      0  66666 133333  96000 -66666 586666]]


In [10]:
#Matrice de force locale
P1_local =  np.zeros((6,1), dtype=np.float32)
P1_local[0,0] = 100

#Matrice de force locale
w = 12
P2_local = np.array([[0], [-w*L2/2], [-w*L2**2/12],[0],[-w*L2/2], [w*L2**2/12]])

print(P1_local)

[[100.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]
 [  0.]]


In [11]:
# Matrice de force global
P1_global = np.matmul(np.transpose(rot_elem_1),P1_local)
P2_global = np.matmul(np.transpose(rot_elem_2),P2_local)

P_global = np.zeros((7,1), dtype=np.float32)
P_global[0:6, 0] = P_global[0:6, 0] + P1_global[0:6, 0]
P_global[1:7, 0] = P_global[1:7, 0] + P2_global[0:6, 0]
print(P_global.astype(int))

[[  0]
 [100]
 [-36]
 [-36]
 [  0]
 [-36]
 [ 36]]


In [12]:
#Calcule des déplacements
U = np.linalg.solve(K_global, P_global)
print(U)

[[-0.00139763]
 [ 0.00591974]
 [ 0.00015999]
 [-0.00075659]
 [ 0.00353514]
 [-0.00195999]
 [-0.00058607]]


# Autres façons

In [13]:
LM = np.zeros((6, 3), dtype=int)
LM[:,0] = [0, 0, 1, 2, 3, 4]
LM[:,1] = [2, 3, 4, 5, 6, 7]
LM[:,2] = [5, 6, 7, 0, 0, 0]

#selctionner les indices differents de zeros
print("Indice ")
indice = LM[:,0] !=0

print(indice)
identity_matrix = np.ones((6,6), dtype=bool)

res = (indice[ np.newaxis,:] & identity_matrix) & (indice[:, np.newaxis] & identity_matrix)
print("res")

# print("Matrice k1 global")
# print(mat_elem_global_1.astype(int))
# print(mat_elem_global_1.shape)
# print("Sous Matrice k1 global")

sous_mat_elem_1 = list()
temp_i = 0
temp_j = 0
list_array = list()
for i in range(6):
    tem_j = 0
    
    for j in range(6):
        if res[i,j] == True:
            print(temp_i, temp_j)
            list_array.append(mat_elem_global_1[i,j])
            temp_j +=1 
    
    if res[i,:].any() == True:
        temp_i +=1
        temp_j = 0
        sous_mat_elem_1.append(np.asarray(list_array))
        list_array = list()

sous_mat_elem_1 = np.asarray(sous_mat_elem_1)
print(sous_mat_elem_1.astype(int))

# # addtionnner au composante differentde zeros
# K_global = np.zeros((7, 7), dtype=np.float32)

# #valeur different de zeros
# mask = LM != 0

# #LM[:,0][mask[0,:]]
# sous_indice =  LM[:,0][mask[:,0]]
# sous_indice =  sous_indice - np.ones_like(sous_indice)
# indice = list(sous_indice)

# temp_i, temp_j = 0, 0
# for i in indice:
#     for j in indice:
#         K_global[i,j] =K_global[i,j] + sous_mat_elem_1[temp_i, temp_j]
#         temp_j = temp_j +1
#     temp_j = 0
#     temp_i = temp_i + 1

# K_global


Indice 
[False False  True  True  True  True]
res
0 0
0 1
0 2
0 3
1 0
1 1
1 2
1 3
2 0
2 1
2 2
2 3
3 0
3 1
3 2
3 3
[[320000  96000      0 160000]
 [ 96000  38400      0  96000]
 [     0      0  40000      0]
 [160000  96000      0 320000]]


In [ ]:
#selctionner les indices differents de zeros
indice = LM[:, 1]!=0
sous_mat_elem_1 = mat_elem_global_1[indice].astype(int)

#valeur different de zeros
mask = LM != 0

#LM[:,0][mask[0,:]]
sous_indice =  LM[:,1][mask[:,1]]
sous_indice =  sous_indice - np.ones_like(sous_indice)
indice = list(sous_indice)

temp_i, temp_j = 0, 0
for i in indice:
    for j in indice:
        K_global[i,j] = K_global[i,j] + sous_mat_elem_1[temp_i, temp_j]
        temp_j = temp_j +1
    temp_j = 0
    temp_i = temp_i + 1

K_global

In [17]:
LM = np.zeros((6, 3), dtype=int)
LM[:,0] = [0, 0, 1, 2, 3, 4]
LM[:,1] = [2, 3, 4, 5, 6, 7]
LM[:,2] = [5, 6, 7, 0, 0, 0]

# addtionnner au composante differentde zeros
K_global_2 = np.zeros((7, 7), dtype=np.float32)

K_global_2 = assemblage_matrice_rigidite(LM, 0, K_global_2, mat_elem_global_1)
K_global_2 = assemblage_matrice_rigidite(LM, 1, K_global_2, mat_elem_global_2)
K_global_2 = assemblage_matrice_rigidite(LM, 2, K_global_2, mat_elem_global_3)

print(K_global_2.astype(int))

[[320000  96000      0 160000      0      0      0]
 [ 96000  71733      0  96000 -33333      0      0]
 [     0      0  62222  66666      0 -22222  66666]
 [160000  96000  66666 586666      0 -66666 133333]
 [     0 -33333      0      0  71733      0  96000]
 [     0      0 -22222 -66666      0  62222 -66666]
 [     0      0  66666 133333  96000 -66666 586666]]


In [19]:
U = np.linalg.solve(K_global, P_global)
print(U)

[[-0.00139763]
 [ 0.00591974]
 [ 0.00015999]
 [-0.00075659]
 [ 0.00353514]
 [-0.00195999]
 [-0.00058607]]
